In [8]:
import os
import glob
import pandas as pd
import tqdm
import numpy as np
from astropy.time import Time
from astropy.coordinates import EarthLocation
import h5py
import yaml
from dataclasses import dataclass
import pylab as plt

In [9]:
dpath = "/home/jovyan/daq-data"

obslist = sorted(glob.glob(f"{dpath}/eb-t*"))

In [10]:
eloc = EarthLocation(x=-2561290.83467119, y=5085918.51537833, z=-2864050.87177975, unit='m')

In [11]:
# Define a dataclass to store metadata in
@dataclass
class ObservationMetadata:
    """ Simple dataclass to store observation metadata """
    obs_id: str
    station: str=''	
    mode: str=''
    sub_mode: str=''	
    intent: str=''	
    notes: str=''	
    observer: str=''	
    reference: str=''	
    utc_start: str=''	
    obs_duration: str='' 	
    loop_duration: str=''
    loop_rest_interval: str=''	
    lst_start: float=''
    start_channel: int=''	
    n_channel: int=''	
    start_frequency: float=''	
    obs_bandwidth: float=''	
    samples_per_frame: int=''	
    time_resolution: float=''	
    n_timesteps: int=''	
    tracking: str=''	
    source_name: str=''	
    right_ascension: str=''	
    declination: str=''	
    altitude: float=''	
    azimuth: float=''	
    obs_date: str=''	
    n_files: int=0	
    size_mb: float=0.0	
    qa: str=''

# Convert HDF5 name to a mode and submode
obs_types = {
    'stationbeam_integ': ['beamformer', 'power'],
    'channel_burst': ['channel-voltages', 'sweep'],
    'channel_integ': ['antenna-bandpass', ' '],
    'correlation_burst': ['correlator', 'sweep'], # NOTE: Could be fixed submode too
    'raw_burst': ['adc-samples', 'synchronous'],  # NOTE: Could be asynchronous submode too
}

In [26]:
def get_md_corr(filelist: list, obs_md: ObservationMetadata) -> ObservationMetadata:
    """ Get correlator metadata from filelist. """
    
    bl = [os.path.basename(f) for f in sorted(filelist)]
    dirname = os.path.dirname(filelist[0])
    
    chan_ids, tsteps = [], []
    modestr1_0, modestr2_0, chan_id, dt0, dt1, tstep = bl[0].replace('.hdf5', '').split('_')
    for fn in bl:
        modestr1, modestr2, chan_id, dt0, dt1, tstep = fn.replace('.hdf5', '').split('_')
        chan_ids.append(chan_id)
        tsteps.append(tstep)
        if modestr1_0 != modestr1:
            print(f"WARNING: {obs_md.obs_id} - Mixed HDF5 file types in {dirname}")

    # Get unique channels and sequence IDs (timesteps)
    chans = sorted([int(c) for c in set(chan_ids)])
    tsteps = sorted([int(t) for t in set(tsteps)])
    
    n_chan = len(chans)
    n_step = len(tsteps)
    
    obs_md.n_channel     = n_chan
    obs_md.n_timesteps   = n_step
    obs_md.start_channel = np.min(chans)
    obs_md.start_frequency = 0.78125 * obs_md.start_channel
    obs_md.obs_bandwidth   = 0.78125 * obs_md.n_channel 

    # Figure out if sweep mode (n_step == 1) or fixed mode (n_chan == 1)
    try:
        assert n_chan == 1 or n_step == 1
    except AssertionError:
        raise RuntimeError(f"Cannot determine mode! n_chan {n_chan} n_step {n_step}")

    obs_md.sub_mode = 'fixed' if n_chan == 1 else 'sweep'

    # Find first file, extract metadata
    first_file = f"{modestr1}_{modestr2}_{chans[0]}_{dt0}_{dt1}_{tsteps[0]}.hdf5"
    with h5py.File(os.path.join(dirname, first_file), mode='r') as h:
        t0 = Time(h['sample_timestamps']['data'][0, 0], format='unix')
        obs_md.time_resolution = np.round( h['root'].attrs['tsamp'], 11)
        obs_md.utc_start = t0.iso
        obs_md.n_timesteps *= h['sample_timestamps']['data'].shape[0]

    # Get last timestamp: in sweep mode, use last channel
    try:
        if obs_md.sub_mode == 'sweep':
            last_file = f"{modestr1}_{modestr2}_{chans[-1]}_{dt0}_{dt1}_{tsteps[0]}.hdf5"
            
            with h5py.File(os.path.join(dirname, last_file), mode='r') as h:
                t1 = Time(h['sample_timestamps']['data'][0, 0] + obs_md.time_resolution, format='unix')
                
        # Get last timestamp: in fixed mode, this will be last file in sequence
        else:
            if n_step > 1:
                last_file = f"{modestr1}_{modestr2}_{chans[-1]}_{dt0}_{dt1}_{tsteps[0]}.hdf5"
                
                with h5py.File(os.path.join(dirname, last_file), mode='r') as h:
                    t1 = Time(h['sample_timestamps']['data'][-1, -1] + obs_md.time_resolution, format='unix')
            else:
                with h5py.File(os.path.join(dirname, first_file), mode='r') as h:
                    #print(list(h['root'].attrs.items()))
                    rr = h['root'].attrs
                    t1 = Time(rr['ts_start'] + rr['tsamp'], format='unix')
        
        obs_md.obs_duration = np.round((t1 - t0).sec, 11)

    except FileNotFoundError:
        print(f"ERROR: {obs_md.obs_id} - file not found: {last_file}")
            
    return obs_md


def get_md_adc(filelist: list, obs_md: ObservationMetadata) -> ObservationMetadata:

    with h5py.File(filelist[0], mode='r') as h:
        
        # ADC dataset is always 32768 in size. Here we check if data > 4096 are zeros,
        # which indicates synchronous mode was used
        dcount = np.sum(h['raw_']['data'][:][4096:])
        obs_md.sub_mode = 'synchronous' if dcount == 0 else 'asynchronous'
        obs_md.time_resolution = np.round(1 / 800e6, 11)
        n_samp = 4096 if dcount == 0 else 32768
        obs_md.obs_duration = n_samp * obs_md.time_resolution
        obs_md.n_timesteps  = n_samp
        
    return obs_md

def get_md_power_beam(filelist: list, obs_md: ObservationMetadata) -> ObservationMetadata:

    # This will sort files in time order
    # Files appear to follow stationbeam_integ_0_20240918_11242_0.hdf5 (X and SEQ zero)
    filelist = sorted(filelist)
    dirname = os.path.dirname(filelist[0])

    
    with h5py.File(filelist[0], mode='r') as h:
        obs_md.time_resolution = h['root'].attrs['tsamp']
        obs_md.n_timesteps  = h['root'].attrs['n_blocks']
        obs_md.n_channel    = h['root'].attrs['n_chans']
        
        t0 = h['root'].attrs['ts_start']
        t1 = h['root'].attrs['ts_end']
        
    if len(filelist) > 1:
        with h5py.File(filelist[-1], mode='r') as h:
            t1 = h['root'].attrs['ts_end']
    obs_md.obs_duration = np.round(t1 - t0, 11)
    obs_md.n_timesteps *= len(filelist)
        
    return obs_md

In [6]:
h5list = glob.glob('/home/jovyan/daq-data/eb-t0001-20240806-00003/ska-low-mccs/15/*.hdf5')
get_md_corr(h5list, ObservationMetadata('eb-t0001-20240806-00003'))

ObservationMetadata(obs_id='eb-t0001-20240806-00003', station='', mode='', sub_mode='sweep', intent='', notes='', observer='', reference='', utc_start='2024-08-06 13:57:08.000', obs_duration=388.15165448189, loop_duration='', loop_rest_interval='', lst_start='', start_channel=64, n_channel=98, start_frequency=50.0, obs_bandwidth=76.5625, samples_per_frame='', time_resolution=1.98180864, n_timesteps=2, tracking='', source_name='', right_ascension='', declination='', altitude='', azimuth='', obs_date='', n_files=0, size_mb=0.0, qa='')

In [217]:
fl = glob.glob("/home/jovyan/daq-data/eb-t0001-20241009-00002/ska-low-mccs/32/*.hdf5")
get_md_adc(fl, ObservationMetadata('eb-t0001-20241009-00002'))

ObservationMetadata(obs_id='eb-t0001-20241009-00002', station='', mode='', sub_mode='synchronous', intent='', notes='', observer='', reference='', utc_start='', obs_duration=5.12e-06, loop_duration='', loop_rest_interval='', lst_start='', start_channel='', n_channel='', start_frequency='', obs_bandwidth='', samples_per_frame='', time_resolution=1.25e-09, n_timesteps=4096, tracking='', source_name='', right_ascension='', declination='', altitude='', azimuth='', obs_date='', n_files=0, size_mb=0.0, qa='')

In [313]:
fl = glob.glob("/home/jovyan/daq-data/eb-t0001-20240918-00011/ska-low-mccs/30/*.hdf5")
get_md_power_beam(fl, ObservationMetadata('eb-t0001-20240918-00011'))

ObservationMetadata(obs_id='eb-t0001-20240918-00011', station='', mode='', sub_mode='', intent='', notes='', observer='', reference='', utc_start='', obs_duration=898.32112121582, loop_duration='', loop_rest_interval='', lst_start='', start_channel='', n_channel=384, start_frequency='', obs_bandwidth='', samples_per_frame='', time_resolution=0.28311552, n_timesteps=207, tracking='', source_name='', right_ascension='', declination='', altitude='', azimuth='', obs_date='', n_files=0, size_mb=0.0, qa='')

In [43]:
fl = glob.glob("/home/jovyan/daq-data/eb-t0001-20240807-00003/ska-low-mccs/3/channel_integ_*.hdf5")
h = h5py.File(fl[0], 'r')
list(h['root'].attrs.items())

[('channel_id', 0),
 ('data_mode', ''),
 ('data_type', 'uint16'),
 ('date_time', '20240807_60023'),
 ('n_antennas', 16),
 ('n_baselines', 0),
 ('n_beams', 1),
 ('n_blocks', 1290),
 ('n_chans', 512),
 ('n_pols', 2),
 ('n_samples', 1),
 ('n_stokes', 4),
 ('station_id', 0),
 ('tile_id', 1),
 ('timestamp', 1723048823.1982272),
 ('ts_end', array([0.])),
 ('ts_start', 1723048823.1982272),
 ('tsamp', 0),
 ('type', 2)]

In [29]:
obs_table = []
for obs in tqdm.tqdm(obslist):
    subdirlist = os.listdir(f"{obs}/ska-low-mccs")
    n_subdir = len(subdirlist)
    obs_id = os.path.basename(obs)
    
    
    if n_subdir >= 1:
        obs_md = ObservationMetadata(obs_id)
        if n_subdir >= 2:
            print(f"WARNING: {obs_id} - multiple scan directories") 
        for subdir in subdirlist:
            obspath = f"{obs}/ska-low-mccs/{subdir}"
            h5list = sorted(glob.glob(f"{obspath}/*.hdf5"))
            data_size_MB = 0
                
            if len(h5list) > 0:
                # Compute data volume
                for h5 in h5list:
                    data_size_MB += os.path.getsize(h5) / 1e6
                obs_md.size_mb = np.round(data_size_MB, 2)
                obs_md.n_files = len(h5list)
                
                # Get metadata from HDF5 files
                with h5py.File(h5list[0], 'r') as fh:
                    t = Time(fh['root'].attrs['ts_start'], format='unix', location=eloc)
                    obs_md.utc_start = t.iso
                    #dstr = t.strftime("%Y-%m-%d")
                    #ut = t.unix
                    obs_md.lst_start = np.round(t.sidereal_time('apparent').value, 3)
                    
                    if fh['root'].attrs['station_id'] != 0:
                        obs_md.station = corr_md['station_id']

                # Get metadata from YAML
                yaml_path = f"{obspath}/obs_metadata.yaml"
                if os.path.exists(yaml_path):
                    with open(yaml_path, 'r') as fh:
                        log = yaml.safe_load(fh)
                        obs_md.station         = log.get('station', '')
                        obs_md.intent          = log.get('intent', '')
                        obs_md.observer        = log.get('observer', '')
                        obs_md.reference       = log.get('reference', '')
                        obs_md.notes           = log.get('notes', '')
                        obs_md.qa              = log.get('qa', '')
                        obs_md.tracking        = log.get('tracking', '')
                        obs_md.source_name     = log.get('source_name', '')
                        obs_md.right_ascension = log.get('ra', '')
                        obs_md.declination     = log.get('dec', '')
                        obs_md.altitude        = log.get('alt', '')
                        obs_md.azimuth         = log.get('az', '')
                        
                # Get mode and submode from HDF5 filenam
                obstype = "_".join(os.path.basename(h5list[0]).split('_')[:2])
                mode, sub_mode = obs_types[obstype]

                obs_md.mode = mode
                obs_md.sub_mode = sub_mode

                try:
                    if mode == 'correlator':
                        obs_md = get_md_corr(h5list, obs_md)
                    elif mode == 'adc-samples':
                        obs_md = get_md_adc(h5list, obs_md)
                    elif mode == 'beamformer' and sub_mode == 'power':
                        obs_md = get_md_power_beam(h5list, obs_md)
                    obs_table.append(obs_md)
                except OSError:
                    print(f"ERROR: OSError: {obs_id} {h5list[0]}")

df = pd.DataFrame(obs_table)

df.to_csv('2024-11-08-log.csv', index=False)

  7%|▋         | 54/742 [00:00<00:02, 263.29it/s]

 45%|████▍     | 331/742 [00:01<00:01, 277.14it/s]

ERROR: eb-t0001-20240911-00011 - file not found: stationbeam_integ_249_20240911_68842_0.hdf5


 89%|████████▉ | 659/742 [00:03<00:00, 141.50it/s]

ERROR: OSError: eb-t0001-20241101-00005 /home/jovyan/daq-data/eb-t0001-20241101-00005/ska-low-mccs/71/correlation_burst_100_20241101_14336_0.hdf5


100%|██████████| 742/742 [00:17<00:00, 41.32it/s] 


In [7]:
dsel = df[df['obs_type'].str.contains('correlation_burst')]
dsel = dsel[dsel['n_files'] == 385]